In [ ]:
import openai
import pandas as pd
from tqdm import tqdm

# Set your OpenAI API key
openai.api_key = "my_key"

# Load your dataset
path = "data_with_summarizes2.csv"  # Replace with the path to your dataset
data = pd.read_csv(path)

# Ensure the dataset has appropriate columns
data['cleaned_text'] = data['cleaned_text'].fillna("").astype(str)
data['cleaned_body'] = data['cleaned_body'].fillna("").astype(str)

# Extract unique posts
unique_posts = data['cleaned_text'].unique()

# Dictionary to store results for each post
post_results = {}

# Refined GPT-4 classification function
def classify_comments(post, comments):
    """
    Classifies comments for a given post using GPT-4.
    """
    results = []
    for comment in comments:
        prompt = f"""
        You are an AI tasked with classifying the conceptual agreement between a post and a comment about climate change. The possible labels are:
        - 1: Agree (the comment conceptually agrees with and supports the post's argument)
        - 0: Neutral/Unrelated (the comment neither agrees nor disagrees conceptually but shifts focus or introduces unrelated ideas)
        - -1: Disagree (the comment rejects or challenges the post's argument).

        Example 1:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "I agree, but we should also focus on methane reductions."
        Label: 1 (Agree)

        Example 2:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "Climate change isn't real. This is all a hoax."
        Label: -1 (Disagree)

        Example 3:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "I think renewable energy is great, but it's expensive."
        Label: 0 (Neutral/Unrelated)

        Classify the following post-comment pair:
        Post: "{post}"

        Comment: "{comment}"

        Provide the label (1, 0, or -1) and a brief explanation for your classification.
        """
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role": "system", "content": "You are a helpful assistant for text classification."},
                          {"role": "user", "content": prompt}],
                max_tokens=200,
                temperature=0
            )
            output = response["choices"][0]["message"]["content"].strip()
            label = int(output.split(":")[0].strip())  # Extract label
            explanation = ":".join(output.split(":")[1:]).strip()  # Extract explanation
            results.append((comment, label, explanation))
        except Exception as e:
            print(f"Error processing: {e}")
            results.append((comment, None, str(e)))  # Append error
    return results

# Process unique posts and their associated comments
for post in tqdm(unique_posts, desc="Processing unique posts"):
    # Get all comments associated with the post
    comments = data[data['cleaned_text'] == post]['cleaned_body'].tolist()
    post_results[post] = classify_comments(post, comments)

# Flatten results into a new DataFrame
results_flat = []
for post, results in post_results.items():
    for comment, label, explanation in results:
        results_flat.append({'cleaned_text': post, 'cleaned_body': comment, 'agreement_label': label, 'label_explanation': explanation})

# Convert to DataFrame
results_df = pd.DataFrame(results_flat)

# Save the results for later use
output_path = "classified_data_high_accuracy.csv"
results_df.to_csv(output_path, index=False)

print(f"Classification completed. Results saved to {output_path}.")


In [ ]:

import pandas as pd
import numpy as np
from tqdm import tqdm
import openai

# Set your OpenAI API key
openai.api_key = "your_api_key_here"

# Load your dataset
path = "data_with_summarizes2.csv"  # Replace with the actual dataset path
data = pd.read_csv(path)

# Ensure the dataset has appropriate columns
data['cleaned_text'] = data['cleaned_text'].fillna("").astype(str)
data['cleaned_body'] = data['cleaned_body'].fillna("").astype(str)

# Shuffle the dataset while keeping track of the original index
shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=False)  # Add original indices as a column
sampled_data = shuffled_data.iloc[:3000].copy()  # Select the first 3,000 rows for labeling

# GPT-4 classification function
def classify_with_gpt(post, comment):
    prompt = f"""
        You are an AI tasked with classifying the conceptual agreement between a post and a comment about climate change. The possible labels are:
        - 1: Agree (the comment conceptually agrees with and supports the post's argument)
        - 0: Neutral/Unrelated (the comment neither agrees nor disagrees conceptually but shifts focus or introduces unrelated ideas)
        - -1: Disagree (the comment rejects or challenges the post's argument).

        Example 1:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "I agree, but we should also focus on methane reductions."
        Label: 1 (Agree)

        Example 2:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "Climate change isn't real. This is all a hoax."
        Label: -1 (Disagree)

        Example 3:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "I think renewable energy is great, but it's expensive."
        Label: 0 (Neutral/Unrelated)

        Classify the following post-comment pair:
        Post: "{post}"

        Comment: "{comment}"

        Provide the label (1, 0, or -1) and a brief explanation for your classification.
        """

  
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "system", "content": "You are a helpful assistant for text classification."},
                      {"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0
        )
        output = response["choices"][0]["message"]["content"].strip()
        label = int(output.split(":")[0].strip())  # Extract label
        explanation = ":".join(output.split(":")[1:]).strip()  # Extract explanation
        return label, explanation
    except Exception as e:
        print(f"Error processing: {e}")
        return None, str(e)  # Append error for debugging

# Apply GPT-4 classification to the sampled data
results = []
for _, row in tqdm(sampled_data.iterrows(), total=len(sampled_data)):
    label, explanation = classify_with_gpt(row['cleaned_text'], row['cleaned_body'])
    results.append((label, explanation))

# Add results to the sampled data
sampled_data['agreement_label'] = [r[0] for r in results]
sampled_data['label_explanation'] = [r[1] for r in results]

# Save the labeled sample data separately
sampled_data_path = "sampled_labeled_data.csv"
sampled_data.to_csv(sampled_data_path, index=False)

# Save the unlabeled data separately
unlabeled_data = shuffled_data.iloc[3000:].copy()  # Rows not selected for labeling
unlabeled_data_path = "unlabeled_data.csv"
unlabeled_data.to_csv(unlabeled_data_path, index=False)

# Combine labeled and unlabeled data while preserving the original order
combined_data = pd.concat([sampled_data, unlabeled_data], ignore_index=True)
combined_data.sort_values('index', inplace=True)  # Reorder by original index
combined_data.drop(columns=['index'], inplace=True)  # Drop the helper index column

# Save the combined dataset with original order
combined_data_path = "data_with_labels.csv"
combined_data.to_csv(combined_data_path, index=False)

print(f"Labeled sample saved to {sampled_data_path}")
print(f"Unlabeled data saved to {unlabeled_data_path}")
print(f"Combined dataset with original order saved to {combined_data_path}")


In [ ]:
def classify_grouped_comments(post, comments, chunk_size=30):
    """
    Classify multiple comments for a single post in chunks to stay within GPT-4's token limit.
    """
    results = []
    for i in range(0, len(comments), chunk_size):
        comment_chunk = comments[i:i + chunk_size]
        prompt = f"""
        You are an AI tasked with classifying the conceptual agreement between a post and a comment about climate change. The possible labels are:
        - 1: Agree (the comment conceptually agrees with and supports the post's argument)
        - 0: Neutral/Unrelated (the comment neither agrees nor disagrees conceptually but shifts focus or introduces unrelated ideas)
        - -1: Disagree (the comment rejects or challenges the post's argument).

        Example 1:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "I agree, but we should also focus on methane reductions."
        Label: 1 (Agree)

        Example 2:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "Climate change isn't real. This is all a hoax."
        Label: -1 (Disagree)

        Example 3:
        Post: "We must reduce CO2 emissions to combat climate change."
        Comment: "I think renewable energy is great, but it's expensive."
        Label: 0 (Neutral/Unrelated)

        Here is the post:
        "{post}"

        Below are the comments to classify:
        {chr(10).join([f"Comment {i+1}: {comment}" for i, comment in enumerate(comment_chunk)])}

        For each comment, provide the label (1, 0, or -1) and a brief explanation for your classification.
        """
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role": "system", "content": "You are a helpful assistant for text classification."},
                          {"role": "user", "content": prompt}],
                max_tokens=500,  # Adjust based on expected output length
                temperature=0
            )
            output = response["choices"][0]["message"]["content"].strip()
            results.extend(output.split("\n"))  # Append results for this chunk
        except Exception as e:
            print(f"Error processing post with comments: {e}")
            results.extend(["Error: Could not process this chunk"] * len(comment_chunk))
    return results


# Process each unique post and its comments
labeled_results = []
for post, group in tqdm(grouped_data, desc="Processing posts"):
    comments = group['cleaned_body'].tolist()  # Get all comments for the post
    raw_output = classify_grouped_comments(post, comments)
    if raw_output:
        # Parse the output to extract labels and explanations
        for comment, response_line in zip(comments, raw_output.split("\n")):
            try:
                label = int(response_line.split(":")[0].strip())  # Extract label
                explanation = ":".join(response_line.split(":")[1:]).strip()  # Extract explanation
                labeled_results.append({
                    'cleaned_text': post,
                    'cleaned_body': comment,
                    'agreement_label': label,
                    'label_explanation': explanation
                })
            except Exception as e:
                labeled_results.append({
                    'cleaned_text': post,
                    'cleaned_body': comment,
                    'agreement_label': None,
                    'label_explanation': f"Error parsing response: {e}"
                })

# Convert labeled results to a DataFrame
labeled_df = pd.DataFrame(labeled_results)

# Save the labeled data
labeled_data_path = "grouped_labeled_sample_data.csv"
labeled_df.to_csv(labeled_data_path, index=False)

print(f"Labeled data saved to {labeled_data_path}")


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import openai
import time

# Set your OpenAI API key
openai.api_key = "your_openai_api_key"

# Load dataset
path = "data_with_summarizes2.csv"  # Replace with the actual dataset path
data = pd.read_csv(path)

# Ensure the dataset has appropriate columns
data['cleaned_text'] = data['cleaned_text'].fillna("").astype(str)
data['cleaned_body'] = data['cleaned_body'].fillna("").astype(str)

# Shuffle the dataset while preserving original indices
shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=False)  # Add original indices as a column

# Select a sample of 3,000 rows for labeling
sampled_data = shuffled_data.iloc[:3000].copy()

# Group comments by post (using cleaned_text or another unique identifier)
grouped_data = sampled_data.groupby('cleaned_text')  # Group by the post content

# Helper function: Chunk large comment groups to stay within GPT-4 limits
def chunk_comments(comments, chunk_size=30):
    """
    Split comments into smaller groups for processing with GPT-4.
    """
    for i in range(0, len(comments), chunk_size):
        yield comments[i:i + chunk_size]

# GPT-4 classification function for grouped comments
def classify_grouped_comments(post, comments, chunk_size=30):
    """
    Classify multiple comments for a single post in chunks to stay within GPT-4's token limit.
    """
    results = []
    print(f"\nProcessing post:\n{post[:300]}...")  # Print first 300 characters of the post
    print(f"Total comments for this post: {len(comments)}\n")  # Print the number of comments

    for comment_chunk in chunk_comments(comments, chunk_size):
        prompt = f"""
        You are an AI tasked with classifying the conceptual agreement between a post and multiple comments about climate change.
        The possible labels for each comment are:
        - 1: Agree (the comment conceptually agrees with and supports the post's argument)
        - 0: Neutral/Unrelated (the comment neither agrees nor disagrees conceptually but shifts focus or introduces unrelated ideas)
        - -1: Disagree (the comment rejects or challenges the post's argument).

        Here is the post:
        "{post}"

        Below are the comments to classify:
        {chr(10).join([f"Comment {i+1}: {comment}" for i, comment in enumerate(comment_chunk)])}

        For each comment, provide the label (1, 0, or -1) and a brief explanation for your classification.
        """
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role": "system", "content": "You are a helpful assistant for text classification."},
                          {"role": "user", "content": prompt}],
                max_tokens=1000,  # Adjust based on expected output length
                temperature=0
            )
            output = response["choices"][0]["message"]["content"].strip()
            print(f"\nRaw GPT-4 response for this chunk:\n{output}\n")  # Print the raw response for this chunk
            results.extend(output.split("\n"))  # Append results for this chunk
            time.sleep(2)  # Pause to avoid rate limits
        except Exception as e:
            print(f"Error processing this chunk: {e}")
            results.extend(["Error: Could not process this chunk"] * len(comment_chunk))
    return results

# Process each unique post and its comments
labeled_results = []
for post, group in tqdm(grouped_data, desc="Processing posts"):
    comments = group['cleaned_body'].tolist()  # Get all comments for the post
    raw_output = classify_grouped_comments(post, comments, chunk_size=30)
    if raw_output:
        # Parse the output to extract labels and explanations
        for comment, response_line in zip(comments, raw_output):
            try:
                label = int(response_line.split(":")[0].strip())  # Extract label
                explanation = ":".join(response_line.split(":")[1:]).strip()  # Extract explanation

                # Debugging output for each comment
                print(f"Post:\n{post[:300]}...")
                print(f"Comment:\n{comment}")
                print(f"Label: {label}")
                print(f"Explanation: {explanation}\n")

                labeled_results.append({
                    'cleaned_text': post,
                    'cleaned_body': comment,
                    'agreement_label': label,
                    'label_explanation': explanation
                })
            except Exception as e:
                print(f"Error parsing response line: {response_line} -> {e}")
                labeled_results.append({
                    'cleaned_text': post,
                    'cleaned_body': comment,
                    'agreement_label': None,
                    'label_explanation': f"Error parsing response: {e}"
                })

# Convert labeled results to a DataFrame
labeled_df = pd.DataFrame(labeled_results)

# Save the labeled sample data
labeled_sample_data_path = "labeled_sample_data.csv"
labeled_df.to_csv(labeled_sample_data_path, index=False)

# Combine labeled data with unlabeled data in the original order
sampled_data = sampled_data.merge(
    labeled_df[['cleaned_text', 'cleaned_body', 'agreement_label']],
    on=['cleaned_text', 'cleaned_body'],
    how='left'
)

# Save the final labeled sampled dataset
final_labeled_data_path = "final_labeled_sampled_data.csv"
sampled_data.to_csv(final_labeled_data_path, index=False)

# Save unlabeled data (remaining rows)
unlabeled_data = shuffled_data.iloc[3000:].copy()
unlabeled_data_path = "unlabeled_data.csv"
unlabeled_data.to_csv(unlabeled_data_path, index=False)

print(f"\nLabeled data saved to {labeled_sample_data_path}")
print(f"Final labeled sampled data saved to {final_labeled_data_path}")
print(f"Unlabeled data saved to {unlabeled_data_path}")


In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline



# Suppress warnings
warnings.filterwarnings("ignore")
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# Load dataset
path = "../newdatasets/datasetsmine/classified_data4.csv"
data = pd.read_csv(path)


# Preprocess data to ensure valid text
data['cleaned_text'] = data['cleaned_text'].fillna("").astype(str)
data['cleaned_body'] = data['cleaned_body'].fillna("").astype(str)



# Initialize models
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased")
semantic_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# Helper function: Extract numbers and keywords
def extract_numbers_and_keywords(text):
    numbers = re.findall(r'\b\d+(\.\d+)?\b', text)  # Extract numbers
    keywords = re.findall(r'\b(growth|percentage|climate|year|auto|market|policy|housing|change|price)\b', text, re.IGNORECASE)
    return set(numbers + keywords)

# Helper function: Calculate cosine similarity using TF-IDF
def calculate_cosine_similarity(post, comment):
    vectorizer = TfidfVectorizer(stop_words="english")
    vectors = vectorizer.fit_transform([post, comment])
    return cosine_similarity(vectors[0:1], vectors[1:2])[0][0]

# Helper function: Calculate semantic similarity using Sentence-BERT
def calculate_semantic_similarity(post, comment):
    post_embedding = semantic_model.encode(post, convert_to_tensor=True)
    comment_embedding = semantic_model.encode(comment, convert_to_tensor=True)
    similarity_score = util.pytorch_cos_sim(post_embedding, comment_embedding).item()
    return similarity_score

# Main classification function
def classify_agreement(post, comment):
    try:
        # Step 1: Sentiment Analysis
        post_sentiment = sentiment_pipeline(post[:512])[0]["label"]
        comment_sentiment = sentiment_pipeline(comment[:512])[0]["label"]
        
        # Step 2: Semantic Similarity
        semantic_score = calculate_semantic_similarity(post, comment)
        
        # Step 3: Cosine Similarity (optional as a double-check)
        cosine_score = calculate_cosine_similarity(post, comment)
        
        # Step 4: Numerical and Keyword Overlap
        post_facts = extract_numbers_and_keywords(post)
        comment_facts = extract_numbers_and_keywords(comment)
        factual_overlap = len(post_facts & comment_facts) > 0
        
        # Step 5: Agreement Logic
        if semantic_score > 0.75 and post_sentiment == comment_sentiment and factual_overlap:
            return 1  # Agree
        elif semantic_score > 0.5 or cosine_score > 0.3 or factual_overlap:
            return 0  # Neutral or Unrelated
        else:
            return -1  # Disagree
    except Exception as e:
        print(f"Error processing post/comment: {e}")
        return 0  # Default to neutral in case of error

# Example Post and Comment
#post = """when you ask the average conservative why they don't seem to take climate change seriously, you often hear something along the lines of well, they've all told us that the world would end in 10 years, and it didn't, and since that didn't happen, my trust is now completely broken and i feel entitled not to have to listen to'experts'anymore. it's this bit about the experts have said the world will end in 10 years that i find highly suspicious, and it is my view that this information likely came from a non-expert, in a casual context, without citation of actual evidence and research."""
#comment = """i dont deny climate change. i just dont want to be priced out of the housing market, auto market, computers i want to enjoy my life."""

# Run the Classification
#label = classify_agreement(post, comment)



# Apply classification function to the dataset
data['agreement_label6'] = data.apply(
    lambda row: classify_agreement(row['cleaned_text'], row['cleaned_body']),
    axis=1
)

# Save the updated dataset
output_path = "../newdatasets/datasetsmine/classified_data6.csv"
data.to_csv(output_path, index=False)

print(f"Classification completed and saved to {output_path}")


# Display the Result
#print(f"Agreement Label: {label}")


In [5]:
#running this code: 



# Set your OpenAI API key
import pandas as pd
import numpy as np
from tqdm import tqdm
import openai
import time

openai.api_key = "my_key"

# Directory path where the thread files are stored
path = "../newdatasets/datasetgroup/data_with_summarizes2.csv"
data = pd.read_csv(path)

# Ensure the dataset has appropriate columns
data['cleaned_text'] = data['cleaned_text'].fillna("").astype(str)
data['cleaned_body'] = data['cleaned_body'].fillna("").astype(str)

# Shuffle the dataset while preserving original indices
#shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=False)  # Add original indices as a column

# Drop the existing 'level_0' column if it exists
if 'level_0' in data.columns:
    data = data.drop(columns=['level_0'])

# Shuffle the dataset while preserving original indices
shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=False)  # Add original indices as a column

# Shuffle the dataset without resetting the index
#shuffled_data = data.sample(frac=1, random_state=42)
#shuffled_data['original_index'] = shuffled_data.index  # Add the original index as a new column



# Select a sample of 3,000 rows for labeling
sampled_data = shuffled_data.iloc[:3000].copy()

# Group comments by post (using cleaned_text or another unique identifier)
grouped_data = sampled_data.groupby('cleaned_text')  # Group by the post content

# Helper function: Chunk large comment groups to stay within GPT-4 limits
def chunk_comments(comments, chunk_size=30):
    """
    Split comments into smaller groups for processing with GPT-4.
    """
    for i in range(0, len(comments), chunk_size):
        yield comments[i:i + chunk_size]

# GPT-4 classification function for grouped comments
def classify_grouped_comments(post, comments, chunk_size=30):
    """
    Classify multiple comments for a single post in chunks to stay within GPT-4's token limit.
    """
    results = []
    print(f"\nProcessing post:\n{post[:300]}...")  # Print first 300 characters of the post
    print(f"Total comments for this post: {len(comments)}\n")  # Print the number of comments

    for comment_chunk in chunk_comments(comments, chunk_size):
        prompt = f"""
        You are an AI tasked with classifying the conceptual agreement between a post and multiple comments about climate change.
        The possible labels for each comment are:
        - 1: Agree (the comment conceptually agrees with and supports the post's argument)
        - 0: Neutral/Unrelated (the comment neither agrees nor disagrees conceptually but shifts focus or introduces unrelated ideas)
        - -1: Disagree (the comment rejects or challenges the post's argument).

        Here is the post:
        "{post}"

        Below are the comments to classify:
        {chr(10).join([f"Comment {i+1}: {comment}" for i, comment in enumerate(comment_chunk)])}

        For each comment, provide the label (1, 0, or -1) and a brief explanation for your classification.
        """
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role": "system", "content": "You are a helpful assistant for text classification."},
                          {"role": "user", "content": prompt}],
                max_tokens=1000,  # Adjust based on expected output length
                temperature=0
            )
            output = response["choices"][0]["message"]["content"].strip()
            print(f"\nRaw GPT-4 response for this chunk:\n{output}\n")  # Print the raw response for this chunk
            results.extend(output.split("\n"))  # Append results for this chunk
            time.sleep(2)  # Pause to avoid rate limits
        except Exception as e:
            print(f"Error processing this chunk: {e}")
            results.extend(["Error: Could not process this chunk"] * len(comment_chunk))
    return results

# Process each unique post and its comments
labeled_results = []
for post, group in tqdm(grouped_data, desc="Processing posts"):
    comments = group['cleaned_body'].tolist()  # Get all comments for the post
    raw_output = classify_grouped_comments(post, comments, chunk_size=30)
    if raw_output:
        # Parse the output to extract labels and explanations
        for comment, response_line in zip(comments, raw_output):
            try:
                label = int(response_line.split(":")[0].strip())  # Extract label
                explanation = ":".join(response_line.split(":")[1:]).strip()  # Extract explanation

                # Debugging output for each comment
                print(f"Post:\n{post[:300]}...")
                print(f"Comment:\n{comment}")
                print(f"Label: {label}")
                print(f"Explanation: {explanation}\n")

                labeled_results.append({
                    'cleaned_text': post,
                    'cleaned_body': comment,
                    'agreement_label': label,
                    'label_explanation': explanation
                })
            except Exception as e:
                print(f"Error parsing response line: {response_line} -> {e}")
                labeled_results.append({
                    'cleaned_text': post,
                    'cleaned_body': comment,
                    'agreement_label': None,
                    'label_explanation': f"Error parsing response: {e}"
                })

# Convert labeled results to a DataFrame
labeled_df = pd.DataFrame(labeled_results)

# Save the labeled sample data
labeled_sample_data_path = "../newdatasets/datasetgroup/labeled_sample_data.csv"
labeled_df.to_csv(labeled_sample_data_path, index=False)

# Combine labeled data with unlabeled data in the original order
sampled_data = sampled_data.merge(
    labeled_df[['cleaned_text', 'cleaned_body', 'agreement_label']],
    on=['cleaned_text', 'cleaned_body'],
    how='left'
)

# Save the final labeled sampled dataset
final_labeled_data_path = "../newdatasets/datasetgroup/final_labeled_sampled_data.csv"
sampled_data.to_csv(final_labeled_data_path, index=False)

# Save unlabeled data (remaining rows)
unlabeled_data = shuffled_data.iloc[3000:].copy()
unlabeled_data_path = "unlabeled_data.csv"
unlabeled_data.to_csv(unlabeled_data_path, index=False)

print(f"\nLabeled data saved to {labeled_sample_data_path}")
print(f"Final labeled sampled data saved to {final_labeled_data_path}")
print(f"Unlabeled data saved to {unlabeled_data_path}")


Processing posts:   0%|                                                                        | 0/234 [00:00<?, ?it/s]


Processing post:
(disclaimer: im not canadian; im american. im sure there are many canadian redditors here in this sub who can cmv on this one and show me what im getting wrong.) the planet will gradually grow hotter and hotter, forcing people to migrate northwards for cooler climates. from a resource standpoint, ca...
Total comments for this post: 6


Raw GPT-4 response for this chunk:
Comment 1: Label 0 - Neutral/Unrelated. This comment is about the delta system on Reddit and does not address the post's argument about climate change and Canada's potential future.

Comment 2: Label 0 - Neutral/Unrelated. This comment seems to be referring to temperature but does not directly address the post's argument about climate change and Canada's potential future.

Comment 3: Label -1 - Disagree. This comment disagrees with the post's argument by suggesting that coastal areas will remain cool due to fog, implying that migration to northern regions like Canada may not be necessary.

Comment 4: L

Processing posts:   0%|▎                                                               | 1/234 [00:13<50:42, 13.06s/it]

Error parsing response line: Comment 1: Label 0 - Neutral/Unrelated. This comment is about the delta system on Reddit and does not address the post's argument about climate change and Canada's potential future. -> invalid literal for int() with base 10: 'Comment 1'
Error parsing response line:  -> invalid literal for int() with base 10: ''
Error parsing response line: Comment 2: Label 0 - Neutral/Unrelated. This comment seems to be referring to temperature but does not directly address the post's argument about climate change and Canada's potential future. -> invalid literal for int() with base 10: 'Comment 2'
Error parsing response line:  -> invalid literal for int() with base 10: ''
Error parsing response line: Comment 3: Label -1 - Disagree. This comment disagrees with the post's argument by suggesting that coastal areas will remain cool due to fog, implying that migration to northern regions like Canada may not be necessary. -> invalid literal for int() with base 10: 'Comment 3'
Er

Processing posts:   1%|▌                                                             | 2/234 [01:17<2:47:50, 43.41s/it]

...

Processing posts: 100%|███████████████████████████████████████████████████████████▋| 233/234 [1:55:47<00:25, 25.19s/it]

Error parsing response line: Comment 1: 0 - Neutral/Unrelated. This comment doesn't directly address the post's argument about climate change or nuclear weapons. It introduces a new concept (agi) without explaining its relevance. -> invalid literal for int() with base 10: 'Comment 1'
Error parsing response line:  -> invalid literal for int() with base 10: ''
Error parsing response line: Comment 2: -1 - Disagree. This comment challenges the post's argument by suggesting that the odds of catastrophic climate change are higher than the odds of a nuclear conflict. It emphasizes that climate change is already happening and could lead to societal collapse, which directly contradicts the post's assertion that climate change is a manageable threat. -> invalid literal for int() with base 10: 'Comment 2'
Error parsing response line:  -> invalid literal for int() with base 10: ''
Error parsing response line: Comment 3: 1 - Agree. This comment supports the post's argument by suggesting that techno

Processing posts: 100%|████████████████████████████████████████████████████████████| 234/234 [1:56:03<00:00, 29.76s/it]

Error parsing response line: Comment 1: 1 - This comment agrees with the post's argument, suggesting that people who live off the ocean and hunt from it are not necessarily doing moral wrong, which aligns with the post's viewpoint. -> invalid literal for int() with base 10: 'Comment 1'
Error parsing response line:  -> invalid literal for int() with base 10: ''
Error parsing response line: Comment 2: 0 - This comment is neutral and unrelated to the main argument of the post. It discusses the effects of peeing on the reef and sewage discharge, which is not a topic directly addressed in the post. -> invalid literal for int() with base 10: 'Comment 2'
Error parsing response line:  -> invalid literal for int() with base 10: ''
Error parsing response line: Comment 3: 1 - This comment agrees with the post's argument, stating that there is enough fish for people to dive for everyday, which supports the post's viewpoint on sustainable hunting. -> invalid literal for int() with base 10: 'Comment


Labeled data saved to labeled_sample_data.csv
Final labeled sampled data saved to final_labeled_sampled_data.csv
Unlabeled data saved to unlabeled_data.csv
